In general, there are three types of constraints:
- Constraint repair
- Penalty methods
- Feasibility-preserving operations

From an implementation standpoint, constraint repair takes the updated particles at time $t$ and applies some function that modifies the particle positions to be enforce feasibility. Penalty methods instead modify the computed objective score after the particle position update as a function of the new positions and the constraints. Finally, feasibility-preserving operations overload the algebraic operations (addition and multiplication) to guarantee feasibility (e.g., using the Aitchison geometry to enforce sum-to-one constraints). 

In [2]:
include("./src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, PSO, Objectives, Designs
using LinearAlgebra

In [3]:
function unit_hypercube_constraints(d::Int)
    I_d = Matrix{Float64}(I, d, d)
    A = vcat(I_d, -I_d)
    b = vcat(ones(d), zeros(d))
    return A, b
end

unit_hypercube_constraints (generic function with 1 method)

In [4]:
A, b = unit_hypercube_constraints(4)
experiment = Experiments.create(7, 4)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("4" => 4, "1" => 1, "2" => 2, "3" => 3), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; … ; -0.0 -0.0 -1.0 -0.0; -0.0 -0.0 -0.0 -1.0], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]), 7, 4)

## Optimization Context
- Create initializer
- Get an enforcer
- Define the enforcer-specific parameters
- Define the optimization algorithm-specific parameters
- Run the optimization

In [ ]:
# Build constraints and objective
constraints = ConstraintEnforcement.LinearConstraints(A, b)
linear_enforcer = ConstraintEnforcement.LinearEnforcer(constraints)
objective = PSO.create_objective(Objectives.griewank, linear_enforcer)

# Initialize swarm
initializer = Designs.create_initializer(experiment)
swarm = PSO.initialize_swarm(initializer, objective, PSO.default_hyperparams())
context = PSO.create_context(swarm, PSO.aggregate_results(save_world=true))
runner_state, history = PSO.optimize(context)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 5 cols, 16 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-8); columns 0(-5); elements 0(-16) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.0000000000e-01
HiGHS run time      :       

(Main.MetaDoE.PSO.RunnerState(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([4.890820795861512e-10 -2.8134754091343155e-9 … 1.843010243560006e-11 5.723929965635913e-9; 4.0973025851430986e-10 -1.6343691805957997e-9 … 5.059849936427728e-10 1.6459226848324053e-9; … ; 1.8812251984665356e-10 -1.1485825890039111e-9 … -3.227674188045905e-10 1.7888437642823707e-9; 6.52829397996889e-11 -3.5477246693167026e-9 … 2.1251121795836163e-10 4.1570238690363944e-11;;; 1.2131886507764898e-10 -7.624188106593122e-10 … -3.938256555871414e-9 1.2956464351242576e-8; 4.660441444673624e-9 1.3609828295481459e-8 … 1.725223134508773e-9 8.26165935853712e-9; … ; -2.549229715297605e-12 -1.7336899324945752e-9 … -6.134147268160276e-12 1.786426629054632e-9; 1.7713253137517256e-10 -3.0320157769955463e-9 … -2.0328599571681035e-9 8.663677077388639e-10;;; 9.104198607606715e-9 -5.766283171563206e-9 … 6.464666046731431e-10 4.3226079855253046e-10; 1.670607183431635e-8 -5.398298962823225e-9 … -1.498689265236953e-8 6.17516

In [7]:
optimizer, optimal_score = PSO.get_optimizer(runner_state)
optimal_score

0.0